In [ ]:
import random
import math
from statistics import mean
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
import time

<h1>Particle Swarm Optimization (PSO)</h1>

In [ ]:
def f(x):
    """
    Define a função f(x, y) para o exercício 1
    """
    return (1 - x[0])**2 + 100 * (x[1] - (x[0]**2))**2

In [ ]:
class Particle:
    """
    Representa uma partícula em um algoritmo de otimização por enxame de partículas (PSO).
    Cada partícula possui uma posição e uma velocidade, que são atualizadas a cada iteração do algoritmo. A partícula também mantém o registro da sua melhor posição encontrada até o momento.

    Atributos:
        - position (list): A posição atual da partícula em um espaço n-dimensional.
        - velocity (list): A velocidade atual da partícula em um espaço n-dimensional.
        - best_position (list): A melhor posição encontrada pela partícula até o momento.
        - fitness (float): O valor da função objetivo avaliada na posição atual da partícula.

    Métodos:
        - evaluate(objective_function): Avalia a função objetivo na posição atual da partícula e atualiza o atributo fitness.
    """
    def __init__(self, n_dimensions, bounds, velocity_range):
        self.position = [random.uniform(bounds[i][0], bounds[i][1]) for i in range(n_dimensions)]
        self.velocity = [random.uniform(velocity_range[0], velocity_range[1]) for i in range(n_dimensions)]
        self.best_position = self.position.copy()
        self.fitness = float('inf')

    def evaluate(self, objective_function):
        # Executa a função objetivo e armazena na propriedade fitness
        self.fitness = objective_function(self.position)
        # Se o valor da função objetivo atual for menor que o melhor valor encontrado até o momento
        if self.fitness < objective_function(self.best_position):
            # atualiza a melhor posição
            self.best_position = self.position.copy()

In [ ]:
class ParticleSwarmOptimization:
    """
    Implementa o algoritmo de Otimização por Enxame de Partículas (Particle Swarm Optimization - PSO).
    Esta classe implementa o algoritmo PSO, permitindo a otimização de uma função objetivo em um espaço de busca n-dimensional. As propriedades e métodos são:

    Propriedades:
        - n_particles (int): Número de partículas no enxame.
        - n_dimensions (int): Número de dimensões do espaço de busca.
        - bounds (list): Limites inferior e superior para cada dimensão do espaço de busca.
        - velocity_range (tuple): Intervalo de velocidade permitido para as partículas.
        - neighborhood_size (int): Tamanho da vizinhança de cada partícula.
        - objective_function (callable): Função objetivo a ser otimizada.
        - c1 (float): Fator de aceleração cognitiva.
        - c2 (float): Fator de aceleração social.
        - max_iterations (int): Número máximo de iterações do algoritmo.

    Métodos:
        - optimize(): Executa o loop de otimização, atualizando a velocidade e posição de cada partícula com base em sua melhor posição e na melhor posição de seus vizinhos. Retorna a melhor posição e aptidão encontradas.
    """
    def __init__(self, n_particles, n_dimensions, bounds, velocity_range, neighborhood_size, objective_function,
                 c1 = 2.0, c2 = 2.0, max_iterations = 1000):
        self.n_particles = n_particles
        self.n_dimensions = n_dimensions
        self.bounds = bounds
        self.velocity_range = velocity_range
        self.neighborhood_size = neighborhood_size
        self.objective_function = objective_function
        
        self.c1 = c1
        self.c2 = c2
        self.max_iterations = max_iterations

        #Inicia partículas e suas vizinhanças
        self.particles = [Particle(n_dimensions, bounds, velocity_range) for _ in range(n_particles)]
        self.neighborhoods = self.__create_neighborhoods()

    def optimize(self):
        """
        Executa o loop de otimização pelo número máximo de iterações, atualizando a velocidade e posição de cada partícula com base em sua melhor posição e na melhor posição de seus vizinhos. Ao final, retorna a melhor posição e aptidão encontradas.
        
        Parâmetros:
            self (Optimizer): Instância da classe Optimizer que contém as informações necessárias para a otimização.
        
        Retorna:
            tuple: Uma tupla contendo a melhor posição e aptidão encontradas.
        """
        min_fitness_by_iteration = []
        mean_fitness_by_iteration = []

        for _ in range(self.max_iterations):
            # Para cada partícula, enumerate conta de 0 até o número de partículas e associa o index a i
            for i, particle in enumerate(self.particles):
                # Obtém a vizinhança da partícula atual
                neighborhood = self.neighborhoods[i]
                # Encontra a melhor partícula vizinha com base na aptidão
                best_neighbor = min(neighborhood, key=lambda p: p.fitness)

                # Atualiza a velocidade e posição de cada dimensão da partícula
                for j in range(self.n_dimensions):
                    # Gera dois números aleatórios entre 0 e 1 e calcula a nova velocidade da partícula
                    r1, r2 = random.random(), random.random()
                    particle.velocity[j] = (particle.velocity[j] +                                                  #inércia
                                            self.c1 * r1 * (particle.best_position[j] - particle.position[j]) +     # comportamento cognitivo
                                            self.c2 * r2 * (best_neighbor.best_position[j] - particle.position[j])) #comportamento social
                    # Limita a velocidade dentro do intervalo especificado, 
                    particle.velocity[j] = max(self.velocity_range[0], min(self.velocity_range[1], particle.velocity[j]))
                    # Atualiza a posição somando a nova velocidade
                    particle.position[j] += particle.velocity[j]
                    # Limita a posição da partícula dentro dos limites especificados
                    particle.position[j] = max(self.bounds[j][0], min(self.bounds[j][1], particle.position[j]))
                # Avalia a aptidão da partícula com a função objetivo
                particle.evaluate(self.objective_function)
            min_fitness_by_iteration.append(min(self.particles, key=lambda p: p.fitness).fitness)
            mean_fitness_by_iteration.append(mean([p.fitness for p in self.particles]))
        # Encontra a melhor partícula com menor valor de aptidão
        best_particle = min(self.particles, key=lambda p: p.fitness)
        # Retorna a melhor posição e aptidão encontradas
        return best_particle.position, best_particle.fitness, min_fitness_by_iteration, mean_fitness_by_iteration

    def __create_neighborhoods(self):
        """
        Cria uma lista de vizinhanças para cada partícula em um enxame.
        
        Cada partícula tem uma vizinhança composta por um número fixo de partículas próximas a ela no enxame. 
        Essa função cria uma lista de listas, onde cada lista interna representa a vizinhança de uma partícula.
        
        Parâmetros:
            self (Enxame): A instância do enxame que está sendo processada.
        
        Retorna:
            list: Uma lista de listas, onde cada lista interna representa a vizinhança de uma partícula no enxame.
        """
        neighborhoods = []
        for i in range(self.n_particles): # Itera sobre o número de partículas
            neighborhood = [] # Inicializa uma lista vazia para armazenar a vizinhança
            for j in range(self.neighborhood_size): # Itera sobre o tamanho da vizinhança
                # Calcula o índice da partícula vizinha usando a operação módulo
                neighbor_index = (i + j) % self.n_particles 
                # Adiciona a partícula vizinha à lista de vizinhança
                neighborhood.append(self.particles[neighbor_index]) 
            # Adiciona a lista de vizinhança à lista de vizinhanças
            neighborhoods.append(neighborhood)
        return neighborhoods

In [ ]:
m_bf, m_bp_x, m_bp_y = 0, 0, 0

bounds = [(-5.0, 5.0), (-5.0, 5.0)] 
velocity_range = [-1.0, 1.0] 

num_particles = 400
max_iter = 30
neighborhood_size = 5

n_dimensions = len(bounds)

for _ in range(100):
    pso = ParticleSwarmOptimization(num_particles, n_dimensions, bounds, velocity_range, neighborhood_size, f, max_iterations=max_iter, c1=2.05, c2=2.05)
    best_position, best_fitness, min_fits, mean_fits, = pso.optimize()
    m_bf += best_fitness
    m_bp_x += best_position[0]
    m_bp_y += best_position[1]

print(f"Melhor posição: ({m_bp_x/100}, {m_bp_y/100})")
print(f"Melhor aptidão: {m_bf/100}")

In [ ]:
#Gráfico
min_fit_line, = plt.plot(range(len(min_fits)), min_fits, label='Aptidão Mínima')
mean_fit_line, = plt.plot(range(len(mean_fits)), mean_fits, label='Aptidão Média')

# Add labels and title
plt.xlabel('Iteração')
plt.ylabel('Aptidão')
plt.title('Aptidão por Iteração')

# Add legend
plt.legend(handles=[min_fit_line, mean_fit_line])

# Show plot
plt.show()

<h1>Ant Colony Optimization</h1>

In [ ]:
class AntColonyOptimization:
    """
    Implementa o algoritmo de Otimização por Colônia de Formigas (ACO) para resolver o Problema do Caixeiro Viajante (TSP).

    Parâmetros:
    - tsp_file (str): Caminho do arquivo contendo os dados do problema TSP.
    - num_ants (int): Número de formigas a serem utilizadas no algoritmo.
    - alpha (float): Parâmetro que controla a importância do feromônio.
    - beta (float): Parâmetro que controla a importância da heurística.
    - rho (float): Taxa de evaporação do feromônio.
    - max_iterations (int): Número máximo de iterações do algoritmo.

    Métodos:
    - solve(): Executa o algoritmo ACO para encontrar a melhor rota para o problema TSP.
    - _choose_next_node(current_node, unvisited): Escolhe o próximo nó a ser visitado por uma formiga, com base na probabilidade.
    - _update_pheromone(paths, distances): Atualiza a matriz de feromônio com base nas rotas e distâncias encontradas pelas formigas.
    - _open_file(tsp_file): Lê os dados do problema TSP a partir de um arquivo.
    - plot_path(path): Plota a rota representada por um caminho.
    - show_best_paths(best_paths, best_distances): Cria uma animação mostrando as melhores rotas encontradas em cada iteração.
    - plot_iterations(best_distances): Plota o gráfico da melhor distância encontrada a cada iteração.
    """
    def __init__(self, tsp_file, num_ants=50, alpha=1.0, beta=5.0, rho=0.5, max_iterations=100):
        self.num_ants = num_ants
        self.alpha = alpha
        self.beta = beta
        self.rho = rho
        self.max_iterations = max_iterations

        # Inicializa num_nodes e nodes a partir do arquivo .tsp
        self._open_file(tsp_file)

        # Inicializa a matriz de feromônios
        self.pheromone = np.ones((self.num_nodes, self.num_nodes))

        # calcula matriz heurística (1 / distância)
        self.heuristic = np.zeros((self.num_nodes, self.num_nodes))
        for i in range(self.num_nodes):
            for j in range(self.num_nodes):
                if i != j:
                    dx = self.nodes[i][0] - self.nodes[j][0]
                    dy = self.nodes[i][1] - self.nodes[j][1]
                    distance = math.sqrt(dx ** 2 + dy ** 2)
                    self.heuristic[i][j] = 1.0 / distance

    def solve(self):
        """
        Executa o algoritmo de Otimização por Colônia de Formigas (ACO) para resolver o Problema do Caixeiro Viajante (TSP).

        Retorna:
            best_path (list): O melhor caminho encontrado, representado como uma lista de índices de nós.
            best_distance (float): A distância total do melhor caminho encontrado.
            best_paths (list): Uma lista contendo o melhor caminho encontrado em cada iteração.
            best_distances (list): Uma lista contendo a melhor distância encontrada em cada iteração.

        Descrição:
            Este método executa o algoritmo de Otimização por Colônia de Formigas (ACO) para resolver o Problema do Caixeiro Viajante (TSP)
            definido pelos nós lidos do arquivo TSP fornecido durante a inicialização da classe.

            O algoritmo é executado por um número máximo de iterações especificado pelo parâmetro `max_iterations`. Em cada iteração,
            um número de formigas artificiais (definido pelo parâmetro `num_ants`) constrói soluções (caminhos) com base nas informações
            de feromônio e heurística.

            Após cada iteração, a melhor solução encontrada até o momento é atualizada, e a matriz de feromônio é atualizada com base
            nas soluções construídas pelas formigas naquela iteração.

            O método retorna o melhor caminho encontrado, a distância total desse caminho, uma lista contendo o melhor caminho de cada
            iteração e uma lista contendo a melhor distância de cada iteração.

            As listas `best_paths` e `best_distances` podem ser usadas para visualizar a evolução das melhores soluções ao longo das
            iterações, por exemplo, através de animações ou gráficos.
        """
        best_path = None
        best_distance = float('inf')
        best_paths = []  # Salva melhor caminho para cada iteração
        best_distances = []  # Salva menor distância para cada iteração

        for _ in range(self.max_iterations):
            # Constrói soluções (caminhos) pelas formigas
            paths = []
            distances = []
            for ant in range(self.num_ants):
                path = []
                # Seleciona um nó inicial aleatório
                current_node = random.randint(0, self.num_nodes - 1)
                # Adiciona o nó inicial ao caminho
                path.append(current_node)
                # Cria um conjunto com todos os nós
                unvisited = set(range(self.num_nodes))
                # Remove o nó inicial do conjunto de nós não visitados
                unvisited.remove(current_node)
                # Inicializa a distância do caminho como zero
                tour_distance = 0.0

                while unvisited: # enquanto houver nós não visitados
                    # Escolhe o próximo nó a ser visitado
                    next_node = self._choose_next_node(current_node, unvisited)
                    # Adiciona o próximo nó ao caminho
                    path.append(next_node)
                    # Remove o próximo nó dos não visitados
                    unvisited.remove(next_node)
                    # Calcula a diferença nas coordenadas x e y
                    dx = self.nodes[current_node][0] - self.nodes[next_node][0]
                    dy = self.nodes[current_node][1] - self.nodes[next_node][1]
                    # Atualiza a distância do caminho e o nó atual
                    tour_distance += math.sqrt(dx ** 2 + dy ** 2)
                    current_node = next_node

                # Adiciona a distância do último nó ao primeiro nó
                dx = self.nodes[path[-1]][0] - self.nodes[path[0]][0]
                dy = self.nodes[path[-1]][1] - self.nodes[path[0]][1]
                tour_distance += math.sqrt(dx ** 2 + dy ** 2)

                # Adiciona caminho e distância à lista
                paths.append(path)
                distances.append(tour_distance)

            # Atualiza a melhor solução
            min_distance = min(distances)
            min_index = distances.index(min_distance)
            if min_distance < best_distance:
                best_distance = min_distance
                best_path = paths[min_index]

            # Armazena o melhor caminho e distância da iteração atual
            best_paths.append(best_path[:])
            best_distances.append(best_distance)

            # Atualiza a matriz de feromônio
            self._update_pheromone(paths, distances)

        return best_path, best_distance, best_paths, best_distances

    def _choose_next_node(self, current_node, unvisited):
        """
        Escolhe o próximo nó a ser visitado pela formiga, com base nas informações de feromônio e heurística.

        Parâmetros:
            current_node (int): O índice do nó atual.
            unvisited (set): Um conjunto de índices de nós não visitados.

        Retorna:
            int: O índice do próximo nó a ser visitado.

        Descrição:
            Este método implementa a regra de transição de estado da Otimização por Colônia de Formigas (ACO).
            Calcula a probabilidade de cada nó não visitado ser escolhido como o próximo nó a ser visitado
            com base nas informações de feromônio e heurística.

            A probabilidade de escolher um nó é proporcional à quantidade de feromônio na aresta que conecta o nó atual ao nó candidato

            O próximo nó é então selecionado aleatoriamente com base nessas probabilidades calculadas.
        """
        # Calcula as probabilidades para cada nó não visitado, com base no feromônio e na heurística
        probabilities = [self.pheromone[current_node][node] ** self.alpha * self.heuristic[current_node][node] ** self.beta for node in unvisited]
        # Calcula a soma das probabilidades
        total_prob = sum(probabilities)
        # Calcula a probabilidade em porcentagem
        probabilities = [prob / total_prob for prob in probabilities]
        # Seleciona o próximo nó aleatoriamente, com base nas probabilidades
        next_node = np.random.choice(list(unvisited), p=probabilities)
        return next_node

    def _update_pheromone(self, paths, distances):
        """
        Atualiza a matriz de feromônio com base nos caminhos construídos pelas formigas.

        Parâmetros:
            paths (list): Uma lista de caminhos, onde cada caminho é uma lista de índices de nós.
            distances (list): Uma lista de distâncias correspondentes a cada caminho.

        Descrição:
            Este método implementa a regra de atualização de feromônio atualizando a matriz de feromônio com base nos camihos construídos pelas formigas na iteração atual.

            Para cada formiga, o método percorre seu caminho e incrementa a quantidade de feromônio nas arestas
            que fazem parte desse caminho. O incremento é proporcional ao inverso da distância do caminho,
            de modo que caminhos mais curtos recebem mais feromônio.

            Após atualizar o feromônio com base nos caminhos das formigas, o método aplica a evaporação de feromônio,
            multiplicando a matriz de feromônio por (1 - rho), onde rho é a taxa de evaporação de feromônio.
            Isso faz com que o feromônio evapore gradualmente ao longo das iterações, permitindo que as formigas
            explorem novos caminhos.
        """
         # Inicializa uma matriz delta de feromônio
        pheromone_delta = np.zeros((self.num_nodes, self.num_nodes))
        for ant in range(self.num_ants):
            # Obtém caminho e distância por formiga
            path = paths[ant]
            distance = distances[ant]
            for i in range(self.num_nodes):
                for j in range(i + 1, self.num_nodes):
                    # Se o nó j está no caminho após o nó i
                    if j in path[path.index(i) + 1:]:
                        # Incrementa o feromônio na aresta (i, j) e (j, i)
                        pheromone_delta[i][j] += 1.0 / distance
                        pheromone_delta[j][i] += 1.0 / distance
        # Aplica a evaporação de feromônio
        self.pheromone = (1 - self.rho) * self.pheromone + pheromone_delta
    
    def _open_file(self, tsp_file):
        """
        Lê os dados do grafo para o arquivo .tsp

        Parâmetros:
            paths (str): Caminho do arquivo contendo os dados do problema TSP.
        """
        # Abre arquivo
        with open(tsp_file) as f:
            lines = f.readlines()

        # Obtém o número de nós
        self.num_nodes = int(lines[3].split(':')[1])
        self.nodes = []
        # Lê posição dos nós do grafo até o EOF
        for line in lines[6:-1]:
            if line.split()[0] == 'EOF':
                break
            x, y = map(float, line.split()[1:3])
            self.nodes.append((x, y))
    
    def plot_path(self, path):
        """
        Plota caminho passado

        Parâmetros:
            path (list): lista de nós do caminho
        """
        x_coords = [self.nodes[node][0] for node in path]
        y_coords = [self.nodes[node][1] for node in path]

        # Adiciona primeiro nó para fechar loop
        x_coords.append(x_coords[0])
        y_coords.append(y_coords[0])

        line, = plt.plot(x_coords, y_coords, 'ro-')
        return line
    
    def show_best_paths(self, best_paths, best_distances, save_as_gif=True, gif_filename='aco_animation.gif'):
        """
        Cria uma animação mostrando o melhor caminho encontrado em cada iteração.

        Parâmetros:
            best_paths (list): Uma lista de caminhos, onde cada caminho é uma lista de índices de nós.
            best_distances (list): Uma lista de distâncias correspondentes a cada melhor caminho.
            save_as_gif (bool, opcional): Se True, a animação será salva como um arquivo GIF. Padrão é False.
            gif_filename (str, opcional): O nome do arquivo GIF a ser salvo. Padrão é 'animation.gif'.
        """
        fig = plt.figure(figsize=(10, 6))
        ax = fig.add_subplot(111)

        # Plota nós
        x_coords = [node[0] for node in self.nodes]
        y_coords = [node[1] for node in self.nodes]
        ax.scatter(x_coords, y_coords, color='black')

        # Cria animação
        def animate(i):
            ax.clear()
            ax.scatter(x_coords, y_coords, color='black')
            line = self.plot_path(best_paths[i])
            ax.add_line(line)
            ax.set_xlabel('X')
            ax.set_ylabel('Y')
            ax.set_title(f'Melhor Caminho na iteração {i} (Distância: {best_distances[i]:.2f})')

        ani = animation.FuncAnimation(fig, animate, frames=len(best_paths), interval=250, repeat=False)
        if save_as_gif:
            ani.save(gif_filename, writer='pillow', fps=5)
        else:
            plt.show()
    
    def plot_iterations(self, best_distances):
        """
        Plota um gráfico mostrando a evolução da melhor distância ao longo das iterações.

        Parâmetros:
            best_distances (list): Uma lista de distâncias, onde cada elemento representa a melhor distância encontrada em uma iteração.
        """
        plt.figure(figsize=(10, 6))
        plt.plot(range(len(best_distances)), best_distances)
        plt.xlabel('Iteração')
        plt.ylabel('Melhor Distância')
        plt.title('Melhor distância por Iteração')
        plt.show()


In [ ]:
from statistics import mean, stdev
time_ = []
bds_ = []
for i in range(100):
    aco = AntColonyOptimization('berlin52.tsp', num_ants=52, alpha=1, beta=5.0, rho=0.5, max_iterations=100)

    start = time.process_time()
    best_path, best_distance, best_paths, best_distances  = aco.solve()
    end = time.process_time()

    time_.append(end-start)
    bds_.append(best_distance)
print(f"Melhor distância \t média:  %.3f"%mean(bds_), f"\t\t stdev:  %.3f"%stdev(bds_))
print("Tempo de Execução \t média: ", mean(time_), "\t stdev: ", stdev(time_))

In [ ]:
aco.plot_path(best_path)

In [ ]:
aco.show_best_paths(best_paths, best_distances)

In [ ]:
aco.plot_iterations(best_distances)